* creates `exp_timing.csv`
- make and save a dataframe containing the onset TR for each event in the experiment
- average across subjects to get a single common TR that will be used to index functional data

In [1]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt

import sklearn
import brainiak
import nilearn as nl
from nilearn import image, plotting, input_data

from scipy.spatial import distance

from utils import *
pd.options.display.max_rows = 200

In [2]:
def read_logfile(sub_num,task='viewing'):
  """ 
  read psychopy logifle into dataframe
  """
  # initialize dataframe
  df = pd.DataFrame(columns=['tstamp','logdata'])
  df = df.astype({'tstamp':float})
  # open logfile
  fpath = 'data/behav/silvy_buckets/sub%iday2/%i_%s.log'%(100+sub_num,sub_num,task)
  f = open(fpath, "r")
  # loop over logfile rows
  for x in f:
    tstamp,B,C = x.split('\t')
    tstamp = float(tstamp)
#     df.loc[tstamp,'logtype'] = B[:-1]
    df.loc[tstamp,'logdata'] = C[:-1]
    df.loc[tstamp,'tstamp'] = tstamp
  return df

def reindex_and_TR_column(df):
  TR_rate = 1.5
  # first TR
  first_TR_tstamp = df[df.logdata=='Keypress: equal'].iloc[0].tstamp
  # center tstamps on first TR
  df.tstamp = df.tstamp-first_TR_tstamp
  # include TR column
  df['TR'] = np.ceil(df.tstamp/TR_rate)
  df = df.astype({'TR':int})
  # remove negative TRs
  df = df[df.TR>-1]
  # reindex by row number
  df.index = np.arange(len(df))
  return df

In [3]:
""" 
COLLECT THE ONSET TR OF EVERY VIDEO ACROSS ALL SUBJECTS IN A LONG DATAFRAME
"""
vid_strL = ['vid1a','vid1b','vid2','vid3','vid4','vid5',
           'vid1a_q','vid1b_q','vid2_q','vid3_q','vid4_q','vid5_q']

timing_df_row_L = []
for sub_num in ALL_SUB_NS:
  ## read dataframe
  df = read_logfile(sub_num)
  df = reindex_and_TR_column(df)
  ## loop over video strings, extract onset TRs
  for vid_str in vid_strL:
    print('sub',sub_num,vid_str)
    ## extract onset TR for wedding string
    TR_vals = df[df.logdata == '%s: autoDraw = True'%vid_str].TR.values
    if vid_str[-2:]=='_q':
      timing_df_row = pd.DataFrame.from_dict({
        'sub_num':np.repeat(sub_num,2),
        'vid_str':np.repeat(vid_str,2),
        'wed_num':[0,11],
        'onset_TR':TR_vals
      })
    else:
      timing_df_row = pd.DataFrame.from_dict({
        'sub_num':np.repeat(sub_num,10),
        'vid_str':np.repeat(vid_str,10),
        'wed_num':np.arange(1,11),
        'onset_TR':TR_vals
      })
    timing_df_row_L.append(timing_df_row)
    
timing_df_cat = pd.concat(timing_df_row_L)
timing_df_cat.iloc[:200]

sub 3 vid1a
sub 3 vid1b
sub 3 vid2
sub 3 vid3
sub 3 vid4
sub 3 vid5
sub 3 vid1a_q
sub 3 vid1b_q
sub 3 vid2_q
sub 3 vid3_q
sub 3 vid4_q
sub 3 vid5_q
sub 4 vid1a
sub 4 vid1b
sub 4 vid2
sub 4 vid3
sub 4 vid4
sub 4 vid5
sub 4 vid1a_q
sub 4 vid1b_q
sub 4 vid2_q
sub 4 vid3_q
sub 4 vid4_q
sub 4 vid5_q
sub 5 vid1a
sub 5 vid1b
sub 5 vid2
sub 5 vid3
sub 5 vid4
sub 5 vid5
sub 5 vid1a_q
sub 5 vid1b_q
sub 5 vid2_q
sub 5 vid3_q
sub 5 vid4_q
sub 5 vid5_q
sub 6 vid1a
sub 6 vid1b
sub 6 vid2
sub 6 vid3
sub 6 vid4
sub 6 vid5
sub 6 vid1a_q
sub 6 vid1b_q
sub 6 vid2_q
sub 6 vid3_q
sub 6 vid4_q
sub 6 vid5_q
sub 7 vid1a
sub 7 vid1b
sub 7 vid2
sub 7 vid3
sub 7 vid4
sub 7 vid5
sub 7 vid1a_q
sub 7 vid1b_q
sub 7 vid2_q
sub 7 vid3_q
sub 7 vid4_q
sub 7 vid5_q
sub 8 vid1a
sub 8 vid1b
sub 8 vid2
sub 8 vid3
sub 8 vid4
sub 8 vid5
sub 8 vid1a_q
sub 8 vid1b_q
sub 8 vid2_q
sub 8 vid3_q
sub 8 vid4_q
sub 8 vid5_q
sub 9 vid1a
sub 9 vid1b
sub 9 vid2
sub 9 vid3
sub 9 vid4
sub 9 vid5
sub 9 vid1a_q
sub 9 vid1b_q
sub 9 vid2_q
sub

KeyboardInterrupt: 

### MEAN ACROSS SUBJECTS: TR VALUE FOR EACH EVENT OF EACH WEDDING

In [ ]:
timing_df = timing_df_cat.groupby(['wed_num','vid_str']).mean()
timing_df = timing_df.apply(np.floor).astype(int)
timing_df = timing_df.reset_index()
timing_df = timing_df.drop('sub_num',axis=1)

### SHIFTING TO ACOUNT FOR HEMODYNAMIC LAG

In [ ]:
timing_df.onset_TR = timing_df.onset_TR + 3

In [ ]:
timing_df.iloc[:200]

# include offset

In [ ]:
"""
the first 26 seconds are intro, followed by 9 seconds start-event, 
17 seconds campfire or flower (depending on label in pkl), 
23 seconds coin or torch, 24 seconds egg or painting, and remainder gifts. 
"""

## NB CURRENTLY USING 10S AS PLACEHOLDER FOR FINAL EVENT 
TR_rate = 1.5
vid_len_D = {'vid1a':26/TR_rate,'vid1b':9/TR_rate,'vid2':17/TR_rate,
             'vid3':23/TR_rate,'vid4':24/TR_rate,'vid5':10/TR_rate,
             'vid1a_q':26/TR_rate,'vid1b_q':9/TR_rate,'vid2_q':17/TR_rate,
             'vid3_q':23/TR_rate,'vid4_q':24/TR_rate,'vid5_q':10/TR_rate
            }
vid_len_D = {k:np.round(v).astype(int) for k,v in vid_len_D.items()}
vid_len_D

In [ ]:
for vid_str,num_TRs in vid_len_D.items():
  timing_df.loc[timing_df['vid_str']==vid_str,'len_TRs'] = num_TRs
timing_df['offset_TR'] = timing_df['onset_TR'] + timing_df['len_TRs']

In [ ]:
timing_df = timing_df.astype({'onset_TR':int,'len_TRs':int,'offset_TR':int})

In [ ]:
timing_df

In [ ]:
# timing_df.to_csv('exp_timing.csv')